# CS 447 Homework 1 $-$ Language Models & Morphological Transduction
In this homework we will study some traditional appraoches to a few natural language tasks. First, you will build some n-gram language models on a corpus of Wikipedia articles, and then you will design a finite-state transducer for verb conjugation in English.

This notebook is designed to be run in Google Colab. Navigate to colab.research.google.com and upload this notebook. Then follow the instructions in the notebook to do the assignent.

To run the notebook, you will need to connect to a Runtime. For this homework, all you need is a CPU. You can change the runtime by going to <TT>Runtime > Change runtime type</TT> and selecting <TT>None</TT> in the <TT>Hardware Accelerator</TT> field. We encourage you to disconnect from the runtime when you are not using it, as Google Colab can limit your resources if you overuse them.

You can read more about Google Colab at https://research.google.com/colaboratory/faq.html.

We have provided import statements to all of the Python libraries that you will need. <b>Please do not import other libraries, as they are not supported by the autograder.</b>

#Part 1: Language Models [60 points]

Here, you will train some <b>n-gram language models</b> on WikiText-2, a corpus of high-quality Wikipedia articles. The dataset was originally introduced in the following paper: https://arxiv.org/pdf/1609.07843v1.pdf. A raw version of the data can easily be viewed here: https://github.com/pytorch/examples/tree/master/word_language_model/data/wikitext-2.


## Preprocessing the Data

To make your models more robust, it is necessary to perform some basic preprocessing on the corpora. <i>You do not need to edit this code.</i>

* <b>Sentence splitting:</b>&nbsp;&nbsp;&nbsp;&nbsp;In this homework, we are interested in modeling individual sentences, rather than longer chunks of text such as paragraphs or documents. The WikiTest dataset provides paragraphs; thus, we provide a simple method to identify individual sentences by splitting paragraphs at punctuation tokens (".",  "!",  "?").

* <b>Sentence markers:</b>&nbsp;&nbsp;&nbsp;&nbsp;For both training and testing corpora, each sentence must be surrounded by a start-of-sentence (`<s>`) and end-of-sentence marker (`/s`). These markers will allow your models to generate sentences that have realistic beginnings and endings.

* <b>Unknown words:</b>&nbsp;&nbsp;&nbsp;&nbsp;In order to deal with unknown words in the test corpora, all words that do not appear in the vocabulary must be replaced with a special token for unknown words (`<UNK>`) before estimating your models. The WikiText dataset has already done this, and you can read about the method in the paper above. When unknown words are encountered in the test corpus, they should be treated as that special token instead.

We provide you with preprocessing code here, and you should not modify it.

In [5]:
# Constants (feel free to use these in your code, but do not change them)
START = "<s>"   # Start-of-sentence token
END = "</s>"    # End-of-sentence-token
UNK = "<UNK>"   # Unknown word token

In [6]:
### DO NOT EDIT THIS CELL ###

import torchtext
import random
def preprocess(data, vocab=None):
    final_data = []
    lowercase = "abcdefghijklmnopqrstuvwxyz"
    for paragraph in data:
        paragraph = [x if x != '<unk>' else UNK for x in paragraph.split()]
        if vocab is not None:
            paragraph = [x if x in vocab else UNK for x in paragraph]
        if paragraph == [] or paragraph.count('=') >= 2: continue
        sen = []
        prev_punct, prev_quot = False, False
        for word in paragraph:
            if prev_quot:
                if word[0] not in lowercase:
                    final_data.append(sen)
                    sen = []
                    prev_punct, prev_quot = False, False
            if prev_punct:
                if word == '"':
                    prev_punct, prev_quot = False, True
                else:
                    if word[0] not in lowercase:
                        final_data.append(sen)
                        sen = []
                        prev_punct, prev_quot = False, False
            if word in {'.', '?', '!'}: prev_punct = True
            sen += [word]
        if sen[-1] not in {'.', '?', '!', '"'}: continue # Prevent a lot of short sentences
        final_data.append(sen)
    vocab_was_none = vocab is None
    if vocab is None:
        vocab = set()
    for i in range(len(final_data)):
        final_data[i] = [START] + final_data[i] + [END]
        if vocab_was_none:
            for word in final_data[i]:
                vocab.add(word)
    return final_data, vocab

def getDataset():
    dataset = torchtext.datasets.WikiText2(root='.data', split=('train', 'valid'))
    train_dataset, vocab = preprocess(dataset[0])
    test_dataset, _ = preprocess(dataset[1], vocab)

    return train_dataset, test_dataset

train_dataset, test_dataset = getDataset()

Run the next cell to see 10 random sentences of the training data.

In [7]:
random.seed(19920723)
if __name__ == '__main__':
    for x in random.sample(train_dataset, 10):
        print (x)

['<s>', 'A', 'spur', 'on', 'their', 'left', ',', 'leading', 'to', 'Suvla', 'Bay', ',', 'was', 'defended', 'by', 'a', 'Turkish', 'trench', 'system', '.', '</s>']
['<s>', 'The', 'Battle', 'of', 'Hubbardton', 'was', 'an', 'engagement', 'in', 'the', 'Saratoga', 'campaign', 'of', 'the', 'American', 'Revolutionary', 'War', 'fought', 'in', 'the', 'village', 'of', 'Hubbardton', ',', 'Vermont', '.', '</s>']
['<s>', 'Popular', 'was', 'uncertain', 'of', 'the', 'sales', 'potential', 'for', 'the', 'two', 'new', 'titles', 'and', 'decided', 'to', 'publish', 'them', 'under', 'its', 'Fictioneers', 'imprint', ',', 'which', 'was', 'used', 'for', 'lower', '@-@', 'paying', 'magazines', '.', '</s>']
['<s>', 'The', 'disks', 'of', 'most', 'stars', 'are', 'much', 'too', 'small', 'in', 'angular', 'size', 'to', 'be', 'observed', 'with', 'current', 'ground', '@-@', 'based', 'optical', 'telescopes', ',', 'and', 'so', '<UNK>', 'telescopes', 'are', 'required', 'to', 'produce', 'images', 'of', 'these', 'objects', '.'

## The LanguageModel Class

You will implement 4 types of language models: a <b>unigram</b> model, a <b>smoothed unigram</b> model, a <b>bigram</b> model, and a <b>smoothed bigram</b> model. Each of the models is worth 15 points and extends the following base class. <b>You do not need to implement anything in this class</b>; you will instead implement each of the following methods in the relevant subclass:

* <b>`__init__(self, trainCorpus)`</b>: Train the language model on `trainCorpus`. This will involve calculating relative frequency estimates according to the type of model you're implementing.

* <b>`generateSentence(self)`</b>: <b>[5 points]</b> Return a sentence that is generated by the language model. It should be a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>, where each <TT>w<sup>(i)</sup></TT> is a word in your vocabulary (including <TT>&lt;UNK&gt;</TT> but exlcuding <TT>&lt;s&gt;</TT> and <TT>&lt;&sol;s&gt;</TT>). You may assume that <TT>&lt;s&gt;</TT> starts each sentence (with probability $1$). The following words <TT>w<sup>(1)</sup></TT>, ... , <TT>w<sup>(n)</sup></TT>, <TT>&lt;&sol;s&gt;</TT> are generated according to your language model's distribution. Note that the number of words <TT>n</TT> is not fixed; instead, you should stop the sentence as soon as you generate the stop token <TT>&lt;&sol;s&gt;</TT>.

* <b>`getSentenceLogProbability(self, sentence)`</b>: <b>[5 points]</b> Return the <em> logarithm of the probability</em> of <TT>sentence</TT>, which is again a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>. You should use the natural logarithm $-$ that is, the base-<em>e</em> logarithm. See the note below about performing your calculations in log space.

* <b>`getCorpusPerplexity(self, testCorpus)`</b>: <b>[5 points]</b> You need to compute the perplexity (normalized inverse log probability) of `testCorpus` according to your model. For a corpus $W$ with $N$ words and a bigram model, Jurafsky and Martin tells you to compute perplexity as follows: 

$$Perplexity(W) = \Big [ \prod_{i=1}^N \frac{1}{P(w^{(i)}|w^{(i-1)})} \Big ]^{1/N}$$

<b>Implementation Hint:</b> In order to avoid underflow, you will likely need to do all of your calculations in log-space. That is, instead of multiplying probabilities, you should add the logarithms of the probabilities and exponentiate the result:

$$\prod_{i=1}^N P(w^{(i)}|w^{(i-1)}) = \exp\Big (\sum_{i=1}^N \log P(w^{(i)}|w^{(i-1)}) \Big ) $$

Using this property should help you in your implementation of `generateSentence(self)` and `getCorpusPerplexity(self, testCorpus)`.

Feel free to implement helper methods as you wish (either in the base class or in the subclases). <b>But be sure not to change the function signatures of the provided methods</b> (i.e. the function and argument names), or else the autograder will fail.

In [8]:
import math
import random
from collections import defaultdict

class LanguageModel(object):
    def __init__(self, trainCorpus):
        '''
        Initialize and train the model (i.e. estimate the model's underlying probability
        distribution from the training corpus.)
        '''

        ### DO NOT EDIT ###
        return

    def generateSentence(self):
        '''
        Generate a sentence by drawing words according to the model's probability distribution.
        Note: Think about how to set the length of the sentence in a principled way.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement generateSentence in each subclass.")

    def getSentenceLogProbability(self, sentence):
        '''
        Calculate the log probability of the sentence provided. 
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getSentenceProbability in each subclass.")
        
    def getCorpusPerplexity(self, testCorpus):
        '''
        Calculate the perplexity of the corpus provided.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getCorpusPerplexity in each subclass.")

    def printSentences(self, n):
        '''
        Prints n sentences generated by your model.
        '''

        ### DO NOT EDIT ###
        for i in range(n):
            sent = self.generateSentence()
            prob = self.getSentenceLogProbability(sent)
            print('Log Probability:', prob , '\tSentence:',sent)

## <font color='red'>TODO:</font> Unigram Model [15 points]

Here, you will implement each of the 4 functions described above for an <b>unsmoothed unigram</b> model. The probability distribution of a word is given by $\hat P(w)$.

In [9]:
class UnigramModel(LanguageModel):
    
    def __init__(self, trainCorpus):
        
        self.dic = defaultdict(int)
        for i in range(0, len(trainCorpus)):
            for word in trainCorpus[i]:
                if word != START:
                    self.dic[word] += 1

        self.total_count = sum(self.dic.values())

        for token in self.dic:
            self.dic[token] = self.dic[token] / self.total_count

    def generateSentence(self):
        
        sentence = [START]
        
        while True:
            next_word = random.choices(list(self.dic.keys()), weights=list(self.dic.values()), k=1)[0]

            if next_word != END and next_word != START:
                sentence.append(next_word)
                continue
            elif next_word == END:
                sentence.append(next_word)
                break
        
        return sentence

    def getSentenceLogProbability(self, sentence):
        
        lst = []
        for i in sentence:
            if i != START:
                lst.append(math.log(self.dic[i]))

        self.LogProb = sum(lst)
        
        return self.LogProb
        
    def getCorpusPerplexity(self, testCorpus):
        
        lst2 = []
        for i in testCorpus:
            for j in i:
                if j != START:
                    lst2.append(-math.log(self.dic[j]))
                    
        N = len(lst2)
        PP = math.exp(sum(lst2)/N)
        
        return PP


We provide you with a testing function that uses very simple training & test corpora (you could compute probability/perplexity by hand if you wanted to). Note that passing this test does not guarantee you a perfect score in the autograder; this is simply to help you debug your model.

In [10]:
def testModel(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}

    #	Read in the test corpus
    corpus = ["By the Late Classic , a network of <unk> ( <unk> ) linked various parts of the city , running for several kilometres through its urban core .",
    "Few people realize how difficult it was to create Sonic 's graphics engine , which allowed for the incredible rate of speed the game 's known for ."]
    corpus, _ = preprocess(corpus)

    if model_type == "unigram":
       senProb, testPerp = -18.9159206916, 40.3970060507 # These are the correct answers (don't change them!)
       model = UnigramModel(corpus)
    elif model_type == "smoothed-unigram":
       senProb, testPerp = -18.8969788221, 41.0547195671 # These are the correct answers (don't change them!)
       model = SmoothedUnigramModel(corpus)
    elif model_type == "bigram":
       senProb, testPerp = -float('inf'), 1.4018400696 # These are the correct answers (don't change them!)
       model = BigramModel(corpus)
    elif model_type == "smoothed-bigram":
       senProb, testPerp = -16.1336514995, 18.6021115212 # These are the correct answers (don't change them!)
       model = SmoothedBigramModelAD(corpus)       

    print("--- TEST: generateSentence() ---")
    modelSen = model.generateSentence()
    senTestPassed = isinstance(modelSen, list) and len(modelSen) > 1 and isinstance(modelSen[0], str)
    if senTestPassed:
        print ("Test generateSentence() passed!")
    else:
        raise Exception("Test generateSentence() failed; did not return a list of strings...")

    print("\n--- TEST: getSentenceLogProbability(...) ---")
    sen = preprocess(["Sonic was difficult ."])[0][0]
    prob = round(model.getSentenceLogProbability(sen), 10)
    if prob == senProb:
        print ("Test getSentenceProbability(...) passed!")
    else:
        raise Exception("Test getSentenceProbability(...) failed; returned probability of " + str(prob) + ", expected probability of " + str(senProb))

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    perp = round(model.getCorpusPerplexity(corpus), 10)
    if perp == testPerp:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        raise Exception("Test getCorpusPerplexity(...) failed; returned perplexity of " + str(perp) + ", expected perplexity of " + str(testPerp))

if __name__=='__main__':
    testModel('unigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Test getSentenceProbability(...) passed!

--- TEST: getCorpusPerplexity(...) ---
Test getCorpusPerplexity(...) passed!


Now, you can train your model on the full WikiText corpus, and evaluate it on the held-out test set.

In [11]:
def runModel(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}
    # Read the corpora
    if model_type == 'unigram':
        model = UnigramModel(train_dataset)
    elif model_type == 'bigram':
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        model = SmoothedUnigramModel(train_dataset)
    else:
        model = SmoothedBigramModelAD(train_dataset)

    print("--------- 5 sentences from your model ---------")
    model.printSentences(5)

    print ("\n--------- Corpus Perplexities ---------")
    print ("Training Set:", model.getCorpusPerplexity(train_dataset))
    print ("Testing Set:", model.getCorpusPerplexity(test_dataset))

if __name__=='__main__':
    runModel('unigram')

--------- 5 sentences from your model ---------
Log Probability: -141.71653155920504 	Sentence: ['<s>', 'became', 'Alabama', 'World', 'of', ',', 'propositions', 'building', 'made', 'speak', '"', 'affair', 'grow', 'of', 'to', 'm', 'source', 'to', 'instinctively', '</s>']
Log Probability: -30.027290861927142 	Sentence: ['<s>', 'peak', 'Coleman', 'time', '</s>']
Log Probability: -147.7149157144084 	Sentence: ['<s>', '<UNK>', 'Artist', 'road', '.', 'them', 'This', 'D', 'modern', 'song', 'this', 'Innis', 'described', 'a', 'a', 'made', 'rainy', 'a', 'dissenting', 'In', 'the', '</s>']
Log Probability: -348.4672852942277 	Sentence: ['<s>', 'authorities', 'Edward', 'the', 'or', 'serve', 'Gardens', 'to', 'the', 'child', 'media', '.', 'treatment', 'Rocky', 'and', 'forms', 'skyline', 'also', 'unsuitable', "'s", 'south', 'Islay', 'weekly', 'alongside', 'The', 'whatever', 'to', 'are', 'He', 'construction', 'as', 'in', 'predators', 'Ground', 'touched', 'extensively', 'the', 'Briggs', 'there', 'been',

## <font color='red'>TODO:</font> Smoothed Unigram Model [15 points]

Here, you will implement each of the 4 functions described above for a <b>unigram</b> model with <b>Laplace (add-one) smoothing</b>. The probability distribution of a word is given by $P_L(w)$. This type of smoothing takes away some of the probability mass for observed events and assigns it to unseen events.

In order to smooth your model, you will need the number of words in the corpus, $N$, and the number of word types, $S$. The distinction between these is meaningful: $N$ indicates the number of word instances, where $S$ refers to the size of our vocabulary. For example, the sentence <em>the cat saw the dog</em> has four word types (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>dog</em>), but five word tokens (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>the</em>, <em>dog</em>). The token <em>the</em> appears twice in the sentence, but they share the same type <em>the</em>.

If $c(w)$ is the frequency of $w$ in the training data, you can compute $P_L(w)$ as follows:

$$P_L(w)=\frac{c(w)+1}{N+S}$$

In [12]:
class SmoothedUnigramModel(LanguageModel):
    
    def __init__(self, trainCorpus):

        self.dic = defaultdict(int)
        for i in range(0, len(trainCorpus)):
            for word in trainCorpus[i]:
                if word != START:
                    self.dic[word] += 1

        self.N = sum(self.dic.values())
        self.S = len(self.dic)

        self.P = {}
        for token in self.dic:
            self.P[token] = (self.dic[token] + 1) / (self.N + self.S)

    def generateSentence(self):

        sentence = [START]
        
        while True:
            next_word = random.choices(list(self.P.keys()), weights=list(self.P.values()), k=1)[0]

            if next_word != END and next_word != START:
                sentence.append(next_word)
                continue
            elif next_word == END:
                sentence.append(next_word)
                break
        
        return sentence

    def getSentenceLogProbability(self, sentence):

        lst = []
        for i in sentence:
            if i != START:
                lst.append(math.log(self.P[i]))

        LogProb = sum(lst)
        
        return LogProb
        
    def getCorpusPerplexity(self, testCorpus):

        lst2 = []
        for i in testCorpus:
            for j in i:
                if j != START:
                    lst2.append(-math.log(self.P[j]))

        N = len(lst2)
        PP = math.exp(sum(lst2)/N)
        
        return PP

In [13]:
if __name__=='__main__':
    testModel('smoothed-unigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Test getSentenceProbability(...) passed!

--- TEST: getCorpusPerplexity(...) ---
Test getCorpusPerplexity(...) passed!


In [14]:
if __name__=='__main__':
    runModel('smoothed-unigram')

--------- 5 sentences from your model ---------
Log Probability: -246.35384324663184 	Sentence: ['<s>', 'Highway', 'particularly', 'Bob', 'of', 'person', '(', "'", 'suitable', ':', 'for', 'parade', 'the', 'followed', 'executive', ';', 'It', 'and', 'hyper', 'and', 'in', 'this', 'Ltd', 'ratios', 'of', '.', 'commercial', 'State', 'domestic', '<UNK>', ',', 'reel', '.', 'languages', '</s>']
Log Probability: -238.58856620112618 	Sentence: ['<s>', 'the', 'the', 'published', 'illustrate', 'early', 'the', 'after', 'Parvati', 'the', 'significant', 'it', 'in', ',', ';', 'accordion', '<UNK>', '<UNK>', '?', 'In', 'former', 'inferior', 'were', 'originally', 'that', 'included', 'I', 'Quiney', 'in', 'he', 'or', 'a', 'the', 'reinforce', 'area', 'won', '</s>']
Log Probability: -57.41611998849951 	Sentence: ['<s>', 'death', '<UNK>', 'Krayot', '90', 'On', '–', 'an', '</s>']
Log Probability: -83.78302991822355 	Sentence: ['<s>', 'mm', 'and', ',', 'died', '.', 'Coal', 'former', 'a', 'at', 'see', '(', 'featu

## <font color='red'>TODO:</font> Bigram Model [15 points]

Here, you will implement each of the 4 functions described above for an <b>unsmoothed bigram</b> model. The probability distribution of a word is given by $\hat P(w'|w)$. Thus, the probability of $w_i$ is conditioned on $w_{i-1}$.

In [15]:
class BigramModel(LanguageModel):
    
    def __init__(self, trainCorpus):

        self.uni_dic = defaultdict(int)
        self.bi_dic = defaultdict(int)

        for sen in trainCorpus:
            for i in range(len(sen) - 1):
                if i < (len(sen)-1):
                    self.bi_dic[(sen[i], sen[i+1])] += 1
                self.uni_dic[sen[i]] += 1

        self.probs = defaultdict(int)

        for bigram in self.bi_dic:
            word_1 = bigram[0]
            self.probs[bigram] = self.bi_dic[bigram] / self.uni_dic[word_1]


    def generateSentence(self):
        
        sentence = [START]
        w_count = 0
        while True:
            if w_count < 20:
                next_word = random.choices(list(self.probs.keys()), weights=list(self.probs.values()), k=1)[0]
                if next_word[0] != START and next_word[0] != END:
                    if next_word[1] != END:
                        sentence.append(next_word[0])
                        sentence.append(next_word[1])
                        w_count += 1
                        continue
                    elif next_word[1] == END:
                        sentence.append(next_word[0])
                        sentence.append(next_word[1])
                        break
            else:
                sentence.append(END)
                break
        
        return sentence

    
    def getSentenceLogProbability(self, sentence):

        lst = []
        for i in range(0, len(sentence)):
            if i < (len(sentence) - 1) and sentence[i+1]:
                if (sentence[i], sentence[i+1]) in self.probs:
                    lst.append(math.log(self.probs[(sentence[i], sentence[i+1])]))
                else:
                    lst.append(-math.inf)
                    
        sen_prob = sum(lst)
               
        return sen_prob

    
    def getCorpusPerplexity(self, testCorpus):

        dic = defaultdict(int)
        N = 0
        for sen in testCorpus:
            for i in range(0, len(sen)):
                if i < (len(sen)-1):  
                    if (sen[i], sen[i+1]) in self.probs:
                        dic[(sen[i], sen[i+1])] += math.log(self.probs[(sen[i], sen[i+1])])
                        N += 1
                    else:
                        dic[(sen[i], sen[i+1])] += -math.inf
                        N += 1

        sum_log_prob = -(sum(dic.values()) / N)
        PP = math.exp(sum_log_prob)

        return PP

        


In [16]:
if __name__=='__main__':
    testModel('bigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Test getSentenceProbability(...) passed!

--- TEST: getCorpusPerplexity(...) ---
Test getCorpusPerplexity(...) passed!


In [17]:
if __name__=='__main__':
    runModel('bigram')

--------- 5 sentences from your model ---------
Log Probability: -inf 	Sentence: ['<s>', 'drag', 'reduction', 'Rellstab', 'would', 'naturalized', 'as', 'culprits', 'were', "'t", 'believe', '1107', 'AD', 'Pauline', 'Ladiges', 'Ione', 're', 'Maeonius', ',', 'relevant', 'in', 'Tawny', 'nurse', 'Customs', 'House', 'stibnite', '(', 'Molina', 'in', 'Eagle', 'Railroad', 'melt', ',', 'harvest', '.', 'manager', 'Kenny', 'redundant', 'and', 'predate', 'feral', '</s>']
Log Probability: -inf 	Sentence: ['<s>', 'sweeping', ',', 'convinced', 'him', 'navigation', 'channel', 'guarantees', 'requested', 'cushion', '"', 'niece', 'Gemma', 'programs', '.', 'armour', 'would', 'toads', '.', 'captured', 'by', 'appease', 'his', 'solution', '.', 'nurseries', '.', 'gryllotalpa', 'the', 'curfew', 'was', 'gaits', ',', 'steeplechase', 'event', 'Occupational', 'Safety', 'iodine', 'from', 'Pictish', 'metalwork', '</s>']
Log Probability: -inf 	Sentence: ['<s>', 'quirky', '"', 'criticize', 'absolutism', 'limp', '"', 'r

## <font color='red'>TODO:</font> Smoothed Bigram Model [15 points]

Here, you will implement each of the 4 functions described above for a <b>bigram</b> model with <b>absolute discounting</b>. The probability distribution of a word is given by $P_{AD}(w’|w)$.

In order to smooth your model, you need to compute a discounting factor $D$. If $n_k$ is the number of bigrams $w_1w_2$ that appear exactly $k$ times, you can compute $D$ as: 

$$D=\frac{n_1}{n_1+2n_2}$$ 

For each word $w$, you then need to compute the number of bigram types $ww’$ as follows: 

$$S(w)=|\{w’|c(ww’)>0\}|$$ 

where $c(ww’)$ is the frequency of $ww’$ in the training data.

Finally, you can compute $P_{AD}(w’|w)$ as follows: 

$$P_{AD}(w’|w)=\frac{\max \big (c(ww’)-D,0\big )}{c(w)}+\bigg (\frac{D}{c(w)}\cdot S(w) \cdot P_L(w’)\bigg )$$ 

where $c(w)$ is the frequency of $w$ in the training data and $P_L(w’)$ is the Laplace-smoothed unigram probability of $w’$.

In [21]:
class SmoothedBigramModelAD(LanguageModel):
    def __init__(self, trainCorpus):

        self.uni_dic = defaultdict(int)
        self.bi_dic = defaultdict(int)

        # Bigram counts
        for sen in trainCorpus:
            for i in range(len(sen) - 1):
                if i < (len(sen)-1):
                    self.bi_dic[(sen[i], sen[i+1])] += 1

        # Unigram counts
        for i in range(0, len(trainCorpus)):
            for word in trainCorpus[i]:
                self.uni_dic[word] += 1

        # Laplace-smoothed unigram (No BOS)
        self.PL = defaultdict(int)
        self.N = sum(self.uni_dic.values()) - self.uni_dic[START]
        self.S = len(self.uni_dic) - 1

        for token in self.uni_dic:
            if token != START:
                self.PL[token] = (self.uni_dic[token] + 1) / (self.N + self.S)
        
        # D: Discounting Factor
        n1 = 0
        n2 = 0
        for bigram in self.bi_dic:
            if self.bi_dic[bigram] == 1:
                n1 += 1
            elif self.bi_dic[bigram] == 2:
                n2 += 1

        self.D = n1 / (n1 + 2*n2)

        # S: number of unique words that follow word 
        self.S_dic = defaultdict(int)
        for word in self.uni_dic:
            for j in self.bi_dic:
                if j[0] == word:
                    if not self.S_dic[word]:
                        self.S_dic[word] = [j[1]]
                    else:
                        self.S_dic[word].append(j[1])

        # Probability Distribution
        self.probs = defaultdict(int)
        for bigram in self.bi_dic:
            wprime = bigram[1]
            w = bigram[0]
            self.probs[bigram] = (max(self.bi_dic[bigram] - self.D, 0) / self.uni_dic[w]) + ((self.D / self.uni_dic[w]) * len(self.S_dic[w]) * self.PL[wprime])

    def generateSentence(self):

        sentence = [START]
        w_count = 0
        while True:
            if w_count < 20:
                next_word = random.choices(list(self.probs.keys()), weights=list(self.probs.values()), k=1)[0]
                if next_word[0] != START and next_word[0] != END:
                    if next_word[1] != END:
                        sentence.append(next_word[0])
                        sentence.append(next_word[1])
                        w_count += 1
                        continue
                    elif next_word[1] == END:
                        sentence.append(next_word[0])
                        sentence.append(next_word[1])
                        break
            else:
                sentence.append(END)
                break
        
        return sentence


    def getSentenceLogProbability(self, sentence):

        lst = []
        for i in range(0, len(sentence)):
            if i < (len(sentence) - 1):
                w = sentence[i]
                wprime = sentence[i+1]
                lst.append(math.log((max(self.bi_dic[(w, wprime)] - self.D, 0) / self.uni_dic[w]) + ((self.D / self.uni_dic[w]) * len(self.S_dic[w]) * self.PL[wprime])))

        sen_prob = sum(lst)

        return sen_prob
        
    def getCorpusPerplexity(self, testCorpus):

        dic = defaultdict(int)
        N = 0
        for sen in testCorpus:
            for i in range(0, len(sen)):
                if i < (len(sen)-1):  
                    w = sen[i]
                    wprime = sen[i+1]
                    dic[(w, wprime)] += math.log((max(self.bi_dic[(w, wprime)] - self.D, 0) / self.uni_dic[w]) + ((self.D / self.uni_dic[w]) * len(self.S_dic[w]) * self.PL[wprime]))
                    N += 1

        sum_log_prob = -(sum(dic.values()) / N)
        PP = math.exp(sum_log_prob)

        return PP

In [22]:
if __name__=='__main__':
    testModel('smoothed-bigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Test getSentenceProbability(...) passed!

--- TEST: getCorpusPerplexity(...) ---
Test getCorpusPerplexity(...) passed!


In [23]:
if __name__=='__main__':
    runModel('smoothed-bigram')

--------- 5 sentences from your model ---------
Log Probability: -321.0887681980964 	Sentence: ['<s>', 'hall', ',', 'Paganism', 'has', 'Branko', '<UNK>', 'heart', 'damage', '1970s', 'and', 'Ayurveda', 'has', 'nmi', ';', 'Northumbria', ',', 'sire', 'foals', 'Penfold', ',', 'resulted', 'in', 'Keyblade', ',', 'mottled', 'with', 'language', '.', 'tubular', 'boiler', 'improvement', 'over', 'lbw', 'decisions', 'Talk', ',', 'Lakes', ',', 'Đình', 'Diệm', '</s>']
Log Probability: -334.33535783993386 	Sentence: ['<s>', 'BCE', ')', 'Liga', '.', 'mitochondrial', 'DNA', 'arithmetic', ',', 'deficiencies', ',', 'Nf6', '11', 'Drawn', 'Blank', 'piston', '@-@', 'implanted', 'Xe', 'Concept', 'Music', 'diaspora', ',', 'parish', '.', 'Mabillard', "'s", 'fountains', 'from', 'accommodate', 'the', 'Helen', 'Who', 'XXX', 'Corps', 'Hema', 'Malini', '1964', '.', 'voltage', '<UNK>', '</s>']
Log Probability: -342.4347490718076 	Sentence: ['<s>', 'chase', 'continues', 'Twenty20', 'cricket', 'furthermore', 'been', '

## Food for Thought
We provide you some questions to think about. <b>You do not need to answer these questions, but we encourage you to give them some thought.</b>
<ol>
<li>When generating sentences with the unigram model, what controls the length of the generated sentences? How does this differ from the sentences produced by the bigram models?
<li>Consider the probability of the generated sentences according to your models. Do your models assign drastically different probabilities to the different sets of sentences? Why do you think that is?
<li>Look back at the sentences generated using your models. In your opinion, which model produces better / more realistic sentences?
<li>For each of the four models, which test corpus has the highest perplexity? Why?
<li> Why do you think it might be a bad idea to use Laplace (add-one) smoothing for a bigram model? How does the absolute discounting method help?
</ol>

# Part 2: Finite State Transducers [40 points]

Here, you will implement a <b>finite state transducer</b> (FST), which transduces the infinitive form of verbs to the present participle (<em>-ing</em>) form. You will be graded according to how well your transducer performs on a hidden test dataset.

Run the cell below to pull the data you can use to develop your FST.

## Provided Code
We provide you with a class <TT>FST</TT>, which is a module for constructing and evaluating FST's. You shouldn't modify any of this code. Here is a description of the most useful methods of this module:


* <b>`FST(self, initialStateName)`</b>: Instantiate an FST with an initial (non-accepting) state named `initialStateName`

* <b>`addState(self, name, isFinal=False)`</b>: Add a state `name` to the FST; by default, `isFinal=False` and so the state is not an accepting state.

* <b>`addTransition(self, inStateName, inString, outString, outStateName)`</b>: Add a transition between state `inStateName` and state `outStateName`, where both of these states already exist in the FST. The FST can traverse this transition after reading `inString`, and outputs `outString` when it does so.

* <b>`addSetTransition(self, inStateName, inStringSet, outStateName)`</b>: Add a transition between state `inStateName` and state `outStateName` for each character in `inStringSet`. For each transition, the FST outputs the same character it reads.

The cell also contains a method to read the data and a scoring function that you will call in the "Test your FST" section.

<b>Do not edit any of the code in this cell!</b>


In [3]:
### DO NOT EDIT THIS CELL ###

import os
import pandas as pd
from tabulate import tabulate
pdtabulate=lambda df:tabulate(df,headers='keys',tablefmt='psql', showindex=False)

def readVerbFile(file):
    url='https://drive.google.com/uc?id=1_BKoF40-TaqKvQy3H05GwP1w_VkpZDsh'
    df = pd.read_csv(url, header=None)

    return df.values.tolist()

verbs = readVerbFile('verbsList.csv')

def testFST(print_examples = 'all'):
    assert print_examples in {'all', 'incorrect', 'none'}, "print_examples must be 'all', 'incorrect', or 'none'"
    rule_classes = [('0', "0 Rule: Add -ing", 'walk    ==>  walking'), ('1-cons', "1 Rule: Drop final -e after consonant", 'ride    ==>  riding'), ('1-u', "1 Rule: Drop final -e after u", 'argue   ==>  arguing'), ('1*', "1 Exception: Don't drop -e", 'see     ==>  seeing'), ('2-b', "2 Rule: Double -b", 'grab    ==>  grabbing'), ('2-d', "2 Rule: Double -d", 'nod     ==>  nodding'), ('2-g', "2 Rule: Double -g", 'tag     ==>  tagging'), ('2-m', "2 Rule: Double -m", 'slam    ==>  slamming'), ('2-n', "2 Rule: Double -n", 'ban     ==>  banning'), ('2-p', "2 Rule: Double -p", 'stop    ==>  stopping'), ('2-r', "2 Rule: Double -r", 'bar     ==>  barring'), ('2-t', "2 Rule: Double -t", 'set     ==>  setting'), ('2*-b', "2 Exception: Don't double -b", 'climb   ==>  climbing'), ('2*-d', "2 Exception: Don't double -d", 'lead    ==>  leading'), ('2*-g', "2 Exception: Don't double -g", 'swing   ==>  swinging'), ('2*-m', "2 Exception: Don't double -m", 'harm    ==>  harming'), ('2*-n', "2 Exception: Don't double -n", 'clean   ==>  cleaning'), ('2*-p', "2 Exception: Don't double -p", 'leap    ==>  leaping'), ('2*-r', "2 Exception: Don't double -r", 'roar    ==>  roaring'), ('2*-t', "2 Exception: Don't double -t", 'halt    ==>  halting'), ('2*-er', "2 Exception: Verbs ending in -er", 'gather  ==>  gathering'), ('2*-en', "2 Exception: Verbs ending in -en", 'happen  ==>  happening'), ('3', "3 Rule: Change -ie to -y", 'die     ==>  dying')]

    f = buildFST()
    #verbs = [v for v in verbs if v[0] in {'canoe', 'hoe', 'shoe', 'snowshoe', 'tiptoe', 'toe'}]
    myParses = f.parseInputList([x[0] for x in verbs])
    scores, totals, examples = {}, {}, []

    for i in range(len(verbs)):
        lemma, form, clas = verbs[i]
        output = myParses[i]
        scores[clas] = scores.get(clas, 0)
        totals[clas] = totals.get(clas, 0) + 1
        if print_examples == 'all' or print_examples == 'incorrect' and form != output:
            examples += [(lemma, form, output, 'CORRECT' if form == output else 'INCORRECT')]
        if form == output: scores[clas] += 1
    
    if print_examples != 'none' and len(examples) > 0:
        examples = pd.DataFrame.from_records(examples, columns = ['Input', 'Correct Output', 'Returned Output', 'Result'])
        print(pdtabulate(examples))

    data= []
    
    # We use a scoring method that accounts for (1) the fact that the default FST gets many verbs correct
    #                                           (2) the class inbalance among the different rules
    # p_scores is for verbs that the default FST gets correct, q_scores for verbs it gets wrong
    # Each contains a list of accuracies for each category group (for example, all of the examples where rule 2 applies are in one group)
    p_scores, q_scores = {'0': [], '1*': [], '2*': [], '2*-en/er': []}, {'1': [], '2': [], '3': []}
    for clas, msg, ex in rule_classes:
        acc = scores[clas]/totals[clas]
        data += [(msg, ex, scores[clas], totals[clas], 100*acc)]
        if clas == '0': p_scores['0'].append(acc)
        elif clas in {'1-cons', '1-u'}: q_scores['1'].append(acc)
        elif clas == '1*': p_scores['1*'].append(acc)
        elif '2' in clas and '*' not in clas: q_scores['2'].append(acc)
        elif '2*' in clas:
            if 'er' in clas or 'en' in clas: p_scores['2*-en/er'].append(acc)
            else: p_scores['2*'].append(acc)
        elif '3' in clas: q_scores['3'].append(acc)
        else: assert False, 'should not get here'

    p_scores = [sum(v) / len(v) for k, v in p_scores.items()] # Get the average for each category group
    q_scores = [sum(v) / len(v) for k, v in q_scores.items()]

    # Score by averaging the p and q scores, then applying grading formula
    p, q = sum(p_scores) / len(p_scores), sum(q_scores) / len(q_scores)
    final_score = q/2 * (1+p) # Score will be 0 when p is 100% and q is 0%; score will be 100% when p and q are both 100%

    print('\nScorecard:')
    data = pd.DataFrame.from_records(data, columns = ['Category', 'Example', 'Correct', 'Total', 'Accuracy (%)'])
    print(pdtabulate(data))

    print("Overall Score:", str(final_score*100) + '%')

class Transition:
    # string_in
    # string_out
    def __init__(self, inState, inString, outString, outState):
        self.state_in = inState
        self.string_in = inString
        self.string_out = outString
        self.state_out = outState

    def equals(self, t):
        if self.state_in == t.state_in \
        and self.string_in == t.string_in \
        and self.string_out == t.string_out \
        and self.state_out == t.state_out:
            return True
        else:
            return False

class FSTstate:
    # id: an integer ID of the state
    # isFinal: is this a final state?
    def __init__(self, n, isF, fst):
        self.id = n
        self.isFinal = isF
        self.transitions = dict() # map inStrings to a set of all possible transitions
        self.FST = fst

    def addTransition(self, inString, outString, outState):
        newTransition = Transition(self, inString, outString, outState)
        if inString in self.transitions:
            for t in self.transitions[inString]:
                if t.equals(newTransition):
                    return
            self.transitions[inString].add(newTransition)
        else:
            self.transitions[inString] = set([])
            self.transitions[inString].add(newTransition)
    
    def parseInputFromStartState(self, inString):
        parseTuple = ("", self.id)
        parses = []
        (accept, stringParses) = self.parseInput(inString)
        if accept:
            for p in stringParses:
                completeParse = [parseTuple]
                completeParse.extend(p)
                parses.append(completeParse)
        return (accept, parses)

    def parseInput(self, inString):
        parses = []
        isAccepted = True
        
        DEBUG = False
        if DEBUG:
            print("parseInput: state: ", self.id, " parsing: " , inString)
        
        # Case 1: no suffix
        if inString == "":
            epsilonParses = []
            epsilonAccepted = False
            # try all epsilon transitions
            if "" in self.transitions:
                transSet = self.transitions[""]
                for t in transSet:
                    outString = t.string_out
                    toStateID = t.state_out
                    toState = self.FST.allStates[toStateID]
                    parseTuple = (outString, toStateID)
                    (suffixAccepted, suffixParses) = toState.parseInput(inString)
                    if suffixAccepted:
                        epsilonAccepted = True
                        if suffixParses == []: #accepts.
                            parse_s = [parseTuple]
                            epsilonParses.append(parse_s)
                        else:
                            for s in suffixParses:
                                parse_s = [parseTuple]
                                parse_s.extend(s)
                                epsilonParses.append(parse_s)
            # if epsilon is accepted, add all its parses
            if epsilonAccepted:
                parses.extend(epsilonParses)
            # if this is a final state, add an empty parse
            if self.isFinal or parses != []:
                if DEBUG:
                    print("Accepted in state ", self.id)
                return (True, parses)
            else:
                if DEBUG:
                    print("Rejected in state ", self.id)
                return (False, None)
        # case 2: non-empty suffix: there needs to be one suffix that parses!)
        hasAcceptedSuffix = False;
        for i in range(0,len(inString)+1):
            prefix = inString[0:i]
            suffix = inString[i:len(inString)]
            if DEBUG:
                print("\t prefix: \'", prefix, "\' I=", i)
            if prefix in self.transitions:
                if DEBUG:
                     print("\t prefix: ", prefix,  "suffix: ", suffix, "I=", i)
                transSet = self.transitions[prefix]
                for t in transSet:
                    outString = t.string_out
                    toStateID = t.state_out
                    toState = self.FST.allStates[toStateID]
                    parseTuple = (outString, toStateID)
                    (suffixAccepted, suffixParses) = toState.parseInput(suffix)
                    if suffixAccepted:
                        hasAcceptedSuffix = True
                        if suffixParses == []:
                            parse_s = [parseTuple]
                            parses.append(parse_s)
                            thisPrefixParses = True
                        for s in suffixParses:
                            parse_s = [parseTuple]
                            parse_s.extend(s)
                            parses.append(parse_s)
        if hasAcceptedSuffix:
            return (True, parses)
        else:
            return (False, None)
                            


    def printState(self):
        if self.isFinal:
            FINAL = "FINAL"
        else: FINAL = ""
        print("State", self.id, FINAL)
        for inString in self.transitions:
            transList = self.transitions[inString]
            for t in transList:
                print("\t", inString, ":", t.string_out, " => ", t.state_out)

                    


class FST:
    def __init__(self, initialStateName="q0"):
        self.nStates = 0
        self.initState = FSTstate(initialStateName, False, self) 
        self.allStates = dict()
        self.allStates[initialStateName] = self.initState
       
    def addState(self, name, isFinal=False):
        if name in self.allStates:
            print("ERROR addState: state", name, "exists already")
            sys.exit()
        elif len(self.allStates) >= 30:
            print("ERROR addState: you can't have more than 30 states")
            sys.exit()
        else:  
            newState = FSTstate(name, isFinal, self)
            self.allStates[name] = newState

    def addTransition(self, inStateName, inString, outString, outStateName):
        if (len(inString) > 1):
            print("ERROR: addTransition: input string ", inString, " is longer than one character")
            sys.exit()
        if inStateName not in self.allStates:
            print("ERROR: addTransition: state ", inStateName, " does not exist")
            sys.exit()
        if outStateName not in self.allStates:
            print("ERROR: addTransition: state ", outStateName, " does not exist")
            sys.exit()
        inState = self.allStates[inStateName]
        inState.addTransition(inString, outString, outStateName)

    # epsilon:epsilon
    def addEpsilonTransition(self, inStateName, outStateName):
        if inStateName not in self.allStates:
            print("ERROR: addEpsilonTransition: state ", inStateName, " does not exist")
            sys.exit()
        if outStateName not in self.allStates:
            print("ERROR: addEpsilonTransition: state ", outStateName, " does not exist")
            sys.exit()
        if inStateName == outStateName:
            print("ERROR: we don't allow epsilon loops")
            sys.exit()
        inState = self.allStates[inStateName]
        inState.addTransition("", "", outStateName)

    # map every element in inStringSet to itself
    def addSetTransition(self, inStateName, inStringSet, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, s, outStateName)

    # map string to itself
    def addSelfTransition(self, inStateName, inString, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", outStateName, " does not exist")
            sys.exit()
         self.addTransition(inStateName, inString, inString, outStateName)

    # map every element in inStringSet to outString
    def addSetToStringTransition(self, inStateName, inStringSet, outString, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetDummyTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetDummyTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, outString, outStateName)
    
            
    # map every element in inStringSet to outString
    def addSetEpsilonTransition(self, inStateName, inStringSet, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetEpsilonTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetEpsionTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, "", outStateName)
            
    def parseInput(self, inString):
        SHOW_STATES = False#True
        inString = inString.rstrip('\n')
        (canParse, allParses)  = self.initState.parseInputFromStartState(inString)
        allParsesAsString = ""
        if canParse:
            for parse in allParses:
                for tuple in parse:
                    outString, outState = tuple
                    allParsesAsString += outString
                if SHOW_STATES:
                    allParsesAsString += "\t  States: "
                    i = 0
                    for tuple in parse:
                        i += 1
                        outString, outState = tuple
                        allParsesAsString += outState
                        if i < len(parse):
                            allParsesAsString += " => "
                    allParsesAsString += "; "
          
            return True, allParsesAsString
        else:
            return False, "FAIL"

    def printFST(self):
        print("Printing FST", str(self))
        for stateID in self.allStates:
            state = self.allStates[stateID]
            state.printState()

    def parseInputList(self, verbList):
        #with open(fileName, "r") as f:
        nParses = 0
        totalStrings = 0
        res = []
        for verb in verbList:#f:
            totalStrings += 1
            canParse, parse = self.parseInput(verb)
            res += [parse]
            if canParse:
                nParses += 1
        fraction = nParses/totalStrings
        print(nParses, "/", totalStrings, "=", str(fraction*100)+'%', "of examples parsed") 
        return res

## Present Participle Transduction
Here we provide you with a specification for producing the <em>-ing</em> form for English verbs. You may assume that all verb stems are at least three letters long.

In many cases, the <em>-ing</em> form just consists of <TT>stem + ing</TT>:

<ul>
<TT>walk ==> walking&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;fly ==> flying</TT>
</ul>

But there are a number of exceptions. In particular, you will need to implement the following rules:
<ol>
<li><b>Drop the final <em>-e</em>:</b> Drop the final <em>-e</em> <b>if and only if</b> it is preceded by a consonant or by the letter <em>u</em>:
<ul>
<em>Examples</em>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<em>Exceptions</em><br>
<TT>ride ==> riding&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;see ==> seeing</TT><br>
<TT>argue ==> arguing&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;canoe ==> canoeing</TT>
</ul>
<li><b>Double the final consonant:</b> Double a final single <em>-b</em>, <em>-d</em>, <em>-g</em>, <em>-m</em>, <em>-n</em>, <em>-p</em>, <em>-r</em>, <em>-t</em> <b>if and only if</b> it is preceded by a single vowel:
<ul>
<em>Examples</em>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<em>Exceptions</em><br>
<TT>grab ==> grabbing&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;climb ==> climbing</TT><br>
<TT>nod ==> nodding&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;lead ==> leading</TT><br>
<TT>tag ==> tagging&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;swing ==> swinging</TT><br>
<TT>slam ==> slamming&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;harm ==> harming</TT><br>
<TT>ban ==> banning&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;clean ==> cleaning</TT><br>
<TT>stop ==> stopping&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;leap ==> leaping</TT><br>
<TT>bar ==> barring&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;roar ==> roaring</TT><br>
<TT>set ==> setting&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;halt ==> halting</TT><br>
</ul>
<b>Exceptions</b> to this rule are verbs ending in <em>-er</em> or <em>-en</em>:
<ul>
<TT>gather ==> gathering&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;happen ==> happening&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</TT>
</ul>
We are assuming American English spelling for words like <em>labeling</em>. Note also that this is a simplification of the actual spelling rule; our data does not contain verbs like <em>pardon</em>, where this rule does not apply, or verbs like <em>deter</em>, where this rule does apply.
<li> <b>Change the final <em>-ie</em> to <em>-y</em>:</b>
<ul>
<TT>die ==> dying</TT>
</ul>
</ol>


## <font color='red'>TODO:</font> Build Your FST [40 points]

We have provided you with a rudimentary (and buggy) implementation for transducing verbs to the <em>-ing</em> form. Your task is to modify the function <TT>buildFST()</TT> below to produce a better FST. You are free to define your own character sets or other helper methods that you require. 

The FST will finish its analysis for a string after having read its last character. If it accepts the string, it will output the transduced form; if it does not accept it, the FST will output FAIL. Your FST may be non-deterministic (that is, a state may have multiple transitions for the same input character), but each accepted string should only have one analysis. If your FST accepts a string along multiple paths, all of the outputs will be returned and the translation will be marked incorrect: e.g. <TT>appeal ==> appealingappealling</TT>. It will accept a string if it reaches a final state after having read its last character.

In [3]:
import os
import pandas as pd
from tabulate import tabulate
pdtabulate=lambda df:tabulate(df,headers='keys',tablefmt='psql', showindex=False)


def readVerbFile(file):
    url='https://drive.google.com/uc?id=1_BKoF40-TaqKvQy3H05GwP1w_VkpZDsh'
    df = pd.read_csv(url, header=None)

    return df.values.tolist()

verbs = readVerbFile('verbsList.csv')

def testFST(print_examples = 'all'):
    assert print_examples in {'all', 'incorrect', 'none'}, "print_examples must be 'all', 'incorrect', or 'none'"
    rule_classes = [('0', "0 Rule: Add -ing", 'walk    ==>  walking'), ('1-cons', "1 Rule: Drop final -e after consonant", 'ride    ==>  riding'), ('1-u', "1 Rule: Drop final -e after u", 'argue   ==>  arguing'), ('1*', "1 Exception: Don't drop -e", 'see     ==>  seeing'), ('2-b', "2 Rule: Double -b", 'grab    ==>  grabbing'), ('2-d', "2 Rule: Double -d", 'nod     ==>  nodding'), ('2-g', "2 Rule: Double -g", 'tag     ==>  tagging'), ('2-m', "2 Rule: Double -m", 'slam    ==>  slamming'), ('2-n', "2 Rule: Double -n", 'ban     ==>  banning'), ('2-p', "2 Rule: Double -p", 'stop    ==>  stopping'), ('2-r', "2 Rule: Double -r", 'bar     ==>  barring'), ('2-t', "2 Rule: Double -t", 'set     ==>  setting'), ('2*-b', "2 Exception: Don't double -b", 'climb   ==>  climbing'), ('2*-d', "2 Exception: Don't double -d", 'lead    ==>  leading'), ('2*-g', "2 Exception: Don't double -g", 'swing   ==>  swinging'), ('2*-m', "2 Exception: Don't double -m", 'harm    ==>  harming'), ('2*-n', "2 Exception: Don't double -n", 'clean   ==>  cleaning'), ('2*-p', "2 Exception: Don't double -p", 'leap    ==>  leaping'), ('2*-r', "2 Exception: Don't double -r", 'roar    ==>  roaring'), ('2*-t', "2 Exception: Don't double -t", 'halt    ==>  halting'), ('2*-er', "2 Exception: Verbs ending in -er", 'gather  ==>  gathering'), ('2*-en', "2 Exception: Verbs ending in -en", 'happen  ==>  happening'), ('3', "3 Rule: Change -ie to -y", 'die     ==>  dying')]

    f = buildFST()
    #verbs = [v for v in verbs if v[0] in {'canoe', 'hoe', 'shoe', 'snowshoe', 'tiptoe', 'toe'}]
    myParses = f.parseInputList([x[0] for x in verbs])
    scores, totals, examples = {}, {}, []

    for i in range(len(verbs)):
        lemma, form, clas = verbs[i]
        output = myParses[i]
        scores[clas] = scores.get(clas, 0)
        totals[clas] = totals.get(clas, 0) + 1
        if print_examples == 'all' or print_examples == 'incorrect' and form != output:
            examples += [(lemma, form, output, 'CORRECT' if form == output else 'INCORRECT')]
        if form == output: scores[clas] += 1
    
    if print_examples != 'none' and len(examples) > 0:
        examples = pd.DataFrame.from_records(examples, columns = ['Input', 'Correct Output', 'Returned Output', 'Result'])
        print(pdtabulate(examples))

    data= []
    
    # We use a scoring method that accounts for (1) the fact that the default FST gets many verbs correct
    #                                           (2) the class inbalance among the different rules
    # p_scores is for verbs that the default FST gets correct, q_scores for verbs it gets wrong
    # Each contains a list of accuracies for each category group (for example, all of the examples where rule 2 applies are in one group)
    p_scores, q_scores = {'0': [], '1*': [], '2*': [], '2*-en/er': []}, {'1': [], '2': [], '3': []}
    for clas, msg, ex in rule_classes:
        acc = scores[clas]/totals[clas]
        data += [(msg, ex, scores[clas], totals[clas], 100*acc)]
        if clas == '0': p_scores['0'].append(acc)
        elif clas in {'1-cons', '1-u'}: q_scores['1'].append(acc)
        elif clas == '1*': p_scores['1*'].append(acc)
        elif '2' in clas and '*' not in clas: q_scores['2'].append(acc)
        elif '2*' in clas:
            if 'er' in clas or 'en' in clas: p_scores['2*-en/er'].append(acc)
            else: p_scores['2*'].append(acc)
        elif '3' in clas: q_scores['3'].append(acc)
        else: assert False, 'should not get here'

    p_scores = [sum(v) / len(v) for k, v in p_scores.items()] # Get the average for each category group
    q_scores = [sum(v) / len(v) for k, v in q_scores.items()]

    # Score by averaging the p and q scores, then applying grading formula
    p, q = sum(p_scores) / len(p_scores), sum(q_scores) / len(q_scores)
    final_score = q/2 * (1+p) # Score will be 0 when p is 100% and q is 0%; score will be 100% when p and q are both 100%

    print('\nScorecard:')
    data = pd.DataFrame.from_records(data, columns = ['Category', 'Example', 'Correct', 'Total', 'Accuracy (%)'])
    print(pdtabulate(data))

    print("Overall Score:", str(final_score*100) + '%')

class Transition:
    # string_in
    # string_out
    def __init__(self, inState, inString, outString, outState):
        self.state_in = inState
        self.string_in = inString
        self.string_out = outString
        self.state_out = outState

    def equals(self, t):
        if self.state_in == t.state_in \
        and self.string_in == t.string_in \
        and self.string_out == t.string_out \
        and self.state_out == t.state_out:
            return True
        else:
            return False

class FSTstate:
    # id: an integer ID of the state
    # isFinal: is this a final state?
    def __init__(self, n, isF, fst):
        self.id = n
        self.isFinal = isF
        self.transitions = dict() # map inStrings to a set of all possible transitions
        self.FST = fst

    def addTransition(self, inString, outString, outState):
        newTransition = Transition(self, inString, outString, outState)
        if inString in self.transitions:
            for t in self.transitions[inString]:
                if t.equals(newTransition):
                    return
            self.transitions[inString].add(newTransition)
        else:
            self.transitions[inString] = set([])
            self.transitions[inString].add(newTransition)
    
    def parseInputFromStartState(self, inString):
        parseTuple = ("", self.id)
        parses = []
        (accept, stringParses) = self.parseInput(inString)
        if accept:
            for p in stringParses:
                completeParse = [parseTuple]
                completeParse.extend(p)
                parses.append(completeParse)
        return (accept, parses)

    def parseInput(self, inString):
        parses = []
        isAccepted = True
        
        DEBUG = False
        if DEBUG:
            print("parseInput: state: ", self.id, " parsing: " , inString)
        
        # Case 1: no suffix
        if inString == "":
            epsilonParses = []
            epsilonAccepted = False
            # try all epsilon transitions
            if "" in self.transitions:
                transSet = self.transitions[""]
                for t in transSet:
                    outString = t.string_out
                    toStateID = t.state_out
                    toState = self.FST.allStates[toStateID]
                    parseTuple = (outString, toStateID)
                    (suffixAccepted, suffixParses) = toState.parseInput(inString)
                    if suffixAccepted:
                        epsilonAccepted = True
                        if suffixParses == []: #accepts.
                            parse_s = [parseTuple]
                            epsilonParses.append(parse_s)
                        else:
                            for s in suffixParses:
                                parse_s = [parseTuple]
                                parse_s.extend(s)
                                epsilonParses.append(parse_s)
            # if epsilon is accepted, add all its parses
            if epsilonAccepted:
                parses.extend(epsilonParses)
            # if this is a final state, add an empty parse
            if self.isFinal or parses != []:
                if DEBUG:
                    print("Accepted in state ", self.id)
                return (True, parses)
            else:
                if DEBUG:
                    print("Rejected in state ", self.id)
                return (False, None)
        # case 2: non-empty suffix: there needs to be one suffix that parses!)
        hasAcceptedSuffix = False;
        for i in range(0,len(inString)+1):
            prefix = inString[0:i]
            suffix = inString[i:len(inString)]
            if DEBUG:
                print("\t prefix: \'", prefix, "\' I=", i)
            if prefix in self.transitions:
                if DEBUG:
                     print("\t prefix: ", prefix,  "suffix: ", suffix, "I=", i)
                transSet = self.transitions[prefix]
                for t in transSet:
                    outString = t.string_out
                    toStateID = t.state_out
                    toState = self.FST.allStates[toStateID]
                    parseTuple = (outString, toStateID)
                    (suffixAccepted, suffixParses) = toState.parseInput(suffix)
                    if suffixAccepted:
                        hasAcceptedSuffix = True
                        if suffixParses == []:
                            parse_s = [parseTuple]
                            parses.append(parse_s)
                            thisPrefixParses = True
                        for s in suffixParses:
                            parse_s = [parseTuple]
                            parse_s.extend(s)
                            parses.append(parse_s)
        if hasAcceptedSuffix:
            return (True, parses)
        else:
            return (False, None)
                            


    def printState(self):
        if self.isFinal:
            FINAL = "FINAL"
        else: FINAL = ""
        print("State", self.id, FINAL)
        for inString in self.transitions:
            transList = self.transitions[inString]
            for t in transList:
                print("\t", inString, ":", t.string_out, " => ", t.state_out)

                    


class FST:
    def __init__(self, initialStateName="q0"):
        self.nStates = 0
        self.initState = FSTstate(initialStateName, False, self) 
        self.allStates = dict()
        self.allStates[initialStateName] = self.initState
       
    def addState(self, name, isFinal=False):
        if name in self.allStates:
            print("ERROR addState: state", name, "exists already")
            sys.exit()
        elif len(self.allStates) >= 30:
            print("ERROR addState: you can't have more than 30 states")
            sys.exit()
        else:  
            newState = FSTstate(name, isFinal, self)
            self.allStates[name] = newState

    def addTransition(self, inStateName, inString, outString, outStateName):
        if (len(inString) > 1):
            print("ERROR: addTransition: input string ", inString, " is longer than one character")
            sys.exit()
        if inStateName not in self.allStates:
            print("ERROR: addTransition: state ", inStateName, " does not exist")
            sys.exit()
        if outStateName not in self.allStates:
            print("ERROR: addTransition: state ", outStateName, " does not exist")
            sys.exit()
        inState = self.allStates[inStateName]
        inState.addTransition(inString, outString, outStateName)

    # epsilon:epsilon
    def addEpsilonTransition(self, inStateName, outStateName):
        if inStateName not in self.allStates:
            print("ERROR: addEpsilonTransition: state ", inStateName, " does not exist")
            sys.exit()
        if outStateName not in self.allStates:
            print("ERROR: addEpsilonTransition: state ", outStateName, " does not exist")
            sys.exit()
        if inStateName == outStateName:
            print("ERROR: we don't allow epsilon loops")
            sys.exit()
        inState = self.allStates[inStateName]
        inState.addTransition("", "", outStateName)

    # map every element in inStringSet to itself
    def addSetTransition(self, inStateName, inStringSet, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", inStateName, " does not exist")
            return 0 
         if outStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", outStateName, " does not exist")
            return 0
         for s in inStringSet:
            self.addTransition(inStateName, s, s, outStateName)

    # map string to itself
    def addSelfTransition(self, inStateName, inString, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", outStateName, " does not exist")
            sys.exit()
         self.addTransition(inStateName, inString, inString, outStateName)

    # map every element in inStringSet to outString
    def addSetToStringTransition(self, inStateName, inStringSet, outString, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetDummyTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetDummyTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, outString, outStateName)
    
            
    # map every element in inStringSet to outString
    def addSetEpsilonTransition(self, inStateName, inStringSet, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetEpsilonTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetEpsionTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, "", outStateName)
            
    def parseInput(self, inString):
        SHOW_STATES = False#True
        inString = inString.rstrip('\n')
        (canParse, allParses)  = self.initState.parseInputFromStartState(inString)
        allParsesAsString = ""
        if canParse:
            for parse in allParses:
                for tuple in parse:
                    outString, outState = tuple
                    allParsesAsString += outString
                if SHOW_STATES:
                    allParsesAsString += "\t  States: "
                    i = 0
                    for tuple in parse:
                        i += 1
                        outString, outState = tuple
                        allParsesAsString += outState
                        if i < len(parse):
                            allParsesAsString += " => "
                    allParsesAsString += "; "
          
            return True, allParsesAsString
        else:
            return False, "FAIL"

    def printFST(self):
        print("Printing FST", str(self))
        for stateID in self.allStates:
            state = self.allStates[stateID]
            state.printState()

    def parseInputList(self, verbList):
        #with open(fileName, "r") as f:
        nParses = 0
        totalStrings = 0
        res = []
        for verb in verbList:#f:
            totalStrings += 1
            canParse, parse = self.parseInput(verb)
            res += [parse]
            if canParse:
                nParses += 1
        fraction = nParses/totalStrings
        print(nParses, "/", totalStrings, "=", str(fraction*100)+'%', "of examples parsed") 
        return res


###########################
###########################

AZ = set("abcdefghijklmnopqrstuvwxyz")
VOWS = set("aeiou")
CONS = AZ-VOWS
E = set("e")
O = set("o")
RNL = set("rnl")
AIU = set("aiu")

def buildFST():

    ## States
    f = FST("q0") # initial state
    f.addState("q_c")
    f.addState("q_v")

    f.addState("q_vv")
    f.addState("q_vu")
    f.addState("q_vx")
    f.addState("q_vvc")
    f.addState("q_vue")
    f.addState("q_cu")   
    f.addState("q_cue")  
    f.addState("q_ci")
    f.addState("q_cie")
    f.addState("q_ce")
    f.addState("q_cc")
    f.addState("q_cci")
    f.addState("q_c_e")
    f.addState("q_c_e_doub")
    f.addState("q_c_ee")
    f.addState("q_c_e_rnl")
    f.addState("q_c_o")
    f.addState("q_c_o_doub")
    f.addState("q_c_oe")
    f.addState("q_c_aiu")
    f.addState("q_c_aiu_doub")
    f.addState("q_c_aiu_y")
    f.addState("q_c_o_yw")
    f.addState("q_ing")
    f.addState("q_EOW", True) # accepting state    

    ## SetTransitions
    f.addSetTransition("q0", CONS, "q_c")
    f.addSetTransition("q_c", CONS, "q_c")
    f.addSetTransition("q_c", VOWS, "q_v")

    f.addSetTransition("q0", VOWS, "q_v")
    f.addSetTransition("q_v", VOWS, "q_v") 
    f.addSetTransition("q_v", CONS, "q_c") 

    f.addSetTransition("q_v", VOWS, "q_vv") 
    f.addSetTransition("q_vv", CONS, "q_vvc") 
    
    f.addSetTransition("q_c", CONS, "q_cc") 
    f.addSetTransition("q_c", E, "q_c_e") 
    f.addSetTransition("q_c_e", RNL, "q_c_e_rnl")

    f.addSetTransition("q_c", O, "q_c_o") 
    f.addSetTransition("q_c", AIU, "q_c_aiu") 

    ## Transitions
    # cook -> cooking    
    f.addTransition("q_vvc", "", "", "q_ing")   
    # queue -> queuing
    f.addTransition("q_v", "u", "u", "q_vu")
    f.addTransition("q_vu", "e", "", "q_vue")
    f.addTransition("q_vue", "", "", "q_ing")
    # die -> dying
    f.addTransition("q_c", "i", "", "q_ci")
    f.addTransition("q_ci", "e", "", "q_cie")
    f.addTransition("q_cie", "", "y", "q_ing")
    # subdue -> subduing
    f.addTransition("q_c", "u", "u", "q_cu")
    f.addTransition("q_cu", "e", "", "q_cue")
    f.addTransition("q_cue", "", "", "q_ing")    
    # ride -> riding
    f.addTransition("q_c", "e", "", "q_ce") 
    f.addTransition("q_ce", "", "", "q_ing") 
    # build -> building
    f.addTransition("q_cc", "i", "i", "q_cci") 
    f.addTransition("q_cc", "", "", "q_ing") 
    # decree -> decreeing
    f.addTransition("q_c_e", "e", "e", "q_c_ee")
    f.addTransition("q_c_ee", "", "", "q_ing")
    # gather -> gathering, happen -> happening, cancel -> canceling
    f.addTransition("q_c_e_rnl", "", "", "q_ing")
    # begging, wedding, embedding
    f.addTransition("q_c_e", "b", "bb", "q_c_e_doub")
    f.addTransition("q_c_e", "d", "dd", "q_c_e_doub")
    f.addTransition("q_c_e", "g", "gg", "q_c_e_doub")
    f.addTransition("q_c_e", "m", "mm", "q_c_e_doub")
    f.addTransition("q_c_e", "p", "pp", "q_c_e_doub")
    f.addTransition("q_c_e", "t", "tt", "q_c_e_doub")
    f.addTransition("q_c_e_doub", "", "", "q_ing")
    # lobbing, nodding, logging, stopping
    f.addTransition("q_c_o", "b", "bb", "q_c_o_doub")
    f.addTransition("q_c_o", "d", "dd", "q_c_o_doub")
    f.addTransition("q_c_o", "g", "gg", "q_c_o_doub")
    f.addTransition("q_c_o", "n", "nn", "q_c_o_doub")
    f.addTransition("q_c_o", "p", "pp", "q_c_o_doub")
    f.addTransition("q_c_o", "r", "rr", "q_c_o_doub")
    f.addTransition("q_c_o", "t", "tt", "q_c_o_doub")
    f.addTransition("q_c_o_doub", "", "", "q_ing")    
    # tiptoe -> tiptoeing
    f.addTransition("q_c_o", "e", "e", "q_c_oe")   
    f.addTransition("q_c_oe", "", "", "q_ing")
    f.addTransition("q_c_o", "y", "y", "q_c_o_yw")  
    f.addTransition("q_c_o", "w", "w", "q_c_o_yw") 
    f.addTransition("q_c_o_yw", "", "", "q_ing") 
    # slamming, barring
    f.addTransition("q_c_aiu", "b", "bb", "q_c_aiu_doub")
    f.addTransition("q_c_aiu", "d", "dd", "q_c_aiu_doub")
    f.addTransition("q_c_aiu", "g", "gg", "q_c_aiu_doub")
    f.addTransition("q_c_aiu", "m", "mm", "q_c_aiu_doub")
    f.addTransition("q_c_aiu", "n", "nn", "q_c_aiu_doub")
    f.addTransition("q_c_aiu", "p", "pp", "q_c_aiu_doub")
    f.addTransition("q_c_aiu", "r", "rr", "q_c_aiu_doub")
    f.addTransition("q_c_aiu", "t", "tt", "q_c_aiu_doub")
    f.addTransition("q_c_aiu_doub", "", "", "q_ing")
    # delay -> delaying, play -> playing
    f.addTransition("q_c_aiu", "y", "y", "q_c_aiu_y")     
    f.addTransition("q_c_aiu_y", "", "", "q_ing")

    # fix -> fixing
    f.addTransition("q_v", "x", "x", "q_vx")
    f.addTransition("q_vx", "", "", "q_ing")
    
    f.addTransition("q_ing", "", "ing", "q_EOW")

    return f

if __name__ == '__main__':
    testFST(print_examples='incorrect')

521 / 521 = 100.0% of examples parsed

Scorecard:
+---------------------------------------+------------------------+-----------+---------+----------------+
| Category                              | Example                |   Correct |   Total |   Accuracy (%) |
|---------------------------------------+------------------------+-----------+---------+----------------|
| 0 Rule: Add -ing                      | walk    ==>  walking   |        49 |      49 |            100 |
| 1 Rule: Drop final -e after consonant | ride    ==>  riding    |        79 |      79 |            100 |
| 1 Rule: Drop final -e after u         | argue   ==>  arguing   |        12 |      12 |            100 |
| 1 Exception: Don't drop -e            | see     ==>  seeing    |         6 |       6 |            100 |
| 2 Rule: Double -b                     | grab    ==>  grabbing  |        12 |      12 |            100 |
| 2 Rule: Double -d                     | nod     ==>  nodding   |         8 |       8 |            10

## Test your FST
We have provided you with a dataset to help you debug and test your FST. Run the cell below to see what verbs your FST handles correctly and your overall score. To get a high score, you must perform well on each of the rules. Note that the autograder uses the same scoring function on a <b>different</b> dataset with the same type of verbs. If you are interested, you can find the scoring function in the "Provided Code" section above.

The `print_examples` argument of this method may be set to one of the following values:
* `all`: Print your FST's output on all of the words in the dataset.
* `incorrect`: Print your FST's output on only the words your FST gets incorrect.
* `none`: Don't print any of the words (only show the score).

In [4]:
if __name__ == '__main__':
    testFST(print_examples='all')

521 / 521 = 100.0% of examples parsed
+--------------+------------------+-------------------+----------+
| Input        | Correct Output   | Returned Output   | Result   |
|--------------+------------------+-------------------+----------|
| employ       | employing        | employing         | CORRECT  |
| spy          | spying           | spying            | CORRECT  |
| worry        | worrying         | worrying          | CORRECT  |
| talk         | talking          | talking           | CORRECT  |
| fill         | filling          | filling           | CORRECT  |
| cancel       | canceling        | canceling         | CORRECT  |
| hijack       | hijacking        | hijacking         | CORRECT  |
| know         | knowing          | knowing           | CORRECT  |
| crash        | crashing         | crashing          | CORRECT  |
| block        | blocking         | blocking          | CORRECT  |
| wash         | washing          | washing           | CORRECT  |
| mark         | marking

# What You Need to Submit

To submit the assignment, download this notebook as a <TT>.py</TT> file. You can do this by going to <TT>File > Download > Download .py</TT>. Then rename it to `hwk1.py` and submit it to the autograder in Gradescope.